<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split

# **CallBack**

In [39]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.97):
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [40]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

fatal: destination path 'skin-disease-build' already exists and is not an empty directory.


In [43]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"
os.listdir(train_dir)

['Lupus and other Connective Tissue diseases',
 'Acne and Rosacea Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Warts Molluscum and other Viral Infections',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Systemic Disease',
 'Bullous Disease Photos',
 'Melanoma Skin Cancer Nevi and Moles',
 'Herpes HPV and other STDs Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Urticaria Hives',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Vascular Tumors',
 'Nail Fungus and other Nail Disease',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Vasculitis Photos',
 'Seborrheic Keratoses and other Benign Tumors',
 'Atopic Dermatitis Photos',
 'Eczema Photos',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Exanthems and Drug Eruptions']

# **Image Generator**

In [44]:
data_generator = ImageDataGenerator(validation_split = 0.2)

In [45]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(train_dir,
                                                   batch_size=20,
                                                   class_mode='sparse',
                                                   target_size=(150,150))     

# Flow validation images in batches of 20 using test_datagen generator
validation_generator =  test_datagen.flow_from_directory( test_dir,
                                                   batch_size=20,
                                                   class_mode='sparse',
                                                   target_size=(150,150))


Found 12453 images belonging to 23 classes.
Found 792 images belonging to 23 classes.


#**Model**

In [49]:
i = Input(shape = (150,150,3))
x = Conv2D(32,(3,3),activation='relu')(i)
#x = MaxPooling2D((2,2))(x)
#x = BatchNormalization()(x)
x = Conv2D(32,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)
x = Conv2D(64,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = Conv2D(64,(3,3),activation='relu')(x)
#x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Dropout(0.25)(x)
# x = Conv2D(128,(3,3),activation='relu')(x)
# x = BatchNormalization()(x)
# x = Conv2D(128,(3,3),activation='relu')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(23,activation='softmax')(x)

model = Model(i,x)
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 146, 146, 32)      9248      
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 73, 73, 32)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 71, 71, 64)        18496     
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 69, 69, 64)        3692

# **Compile and Run Model**

In [50]:
model.compile(optimizer = RMSprop(lr=0.0001),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(train_generator,validation_data=validation_generator,epochs=3,callbacks=[callbacks])

Epoch 1/30
64/64 [==============================] - 148s 2s/step - loss: 123.9211 - accuracy: 0.0967
Epoch 2/30
64/64 [==============================] - 148s 2s/step - loss: 3.0454 - accuracy: 0.1157
Epoch 3/30
64/64 [==============================] - 148s 2s/step - loss: 2.9601 - accuracy: 0.1075
Epoch 4/30
64/64 [==============================] - 148s 2s/step - loss: 2.9073 - accuracy: 0.1203
Epoch 5/30
64/64 [==============================] - 146s 2s/step - loss: 2.9089 - accuracy: 0.1238
Epoch 6/30
64/64 [==============================] - 146s 2s/step - loss: 2.9244 - accuracy: 0.1092
Epoch 7/30
64/64 [==============================] - 147s 2s/step - loss: 2.8815 - accuracy: 0.1381
Epoch 8/30
64/64 [==============================] - 147s 2s/step - loss: 2.8957 - accuracy: 0.1357
Epoch 9/30
64/64 [==============================] - 147s 2s/step - loss: 2.8147 - accuracy: 0.1595
Epoch 10/30
64/64 [==============================] - 147s 2s/step - loss: 2.8943 - accuracy: 0.1400
Epoch 1

## **GRAPH**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()